In [13]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import itertools

data = pd.read_csv('data_big.csv')

# Разделение данных на независимые переменные и зависимую переменную
independent_vars = data[['SBP.1', 'DBP.1', 'SV.1', 'CO.1']]
dependent_var = data['weakness.1']

# Проверка независимости для каждой независимой переменной
for var in independent_vars.columns:
    contingency_table = pd.crosstab(data[var], dependent_var)
    
    # Критерий хи-квадрат
    chi2_stat, p_value, _, _ = stats.chi2_contingency(contingency_table)
    
#     Точный критерий Фишера
    try:
        oddsratio, p_value_fisher = stats.fisher_exact(contingency_table)
    except:
        p_value_fisher = 'Точный критерий Фишера не может быть рассчитан для этих данных (должно быть по 2 уникальных значения)'

    
    print('Переменная:', var)
    print('Хи-квадрат статистика:', chi2_stat)
    print('p-значение хи-квадрат:', p_value)
    print('Одностороннее p-значение точного критерия Фишера:', p_value_fisher)
    print()

    
def entropy(x):
    p = x / x.sum()
    p = p[p != 0]
    return -sum(p * np.log2(p))

def uncertainty_coefficient(tab):
    Hxy = entropy(tab.values.flatten())
    Hx = entropy(tab.sum(axis=1))
    Hy = entropy(tab.sum(axis=0))
    I = Hx + Hy - Hxy
    return [I / Hx, I / Hy, 2 * I / (Hx + Hy)] * 100


# Коэффициенты неопределенности для пар независимых переменных
X = data[['SBP.1', 'DBP.1']]
y = data['weakness.1']

def bin_encoding(X):
    return X.apply(lambda x: x.shift().fillna(1) * x).sum(axis=1)

def uncertainty_coefficients_multiple(x, X, k):
    C = list(itertools.combinations(X.columns, k))
    L = []
    for z in C:
        encoded = bin_encoding(X[list(z)])
        table = pd.crosstab(x, encoded)
        L.append(uncertainty_coefficient(table)[0])
    df = pd.DataFrame(C, columns=[f"Variable {i+1}" for i in range(k)])
    df["L"] = L
    return df.sort_values(by="L", ascending=False)

Переменная: SBP.1
Хи-квадрат статистика: 38.29722222222222
p-значение хи-квадрат: 0.4559976299540941
Одностороннее p-значение точного критерия Фишера: Точный критерий Фишера не может быть рассчитан для этих данных (должно быть по 2 уникальных значения)

Переменная: DBP.1
Хи-квадрат статистика: 27.91777777777778
p-значение хи-квадрат: 0.7594656985238902
Одностороннее p-значение точного критерия Фишера: Точный критерий Фишера не может быть рассчитан для этих данных (должно быть по 2 уникальных значения)

Переменная: SV.1
Хи-квадрат статистика: 57.25892857142858
p-значение хи-квадрат: 0.22379778219738783
Одностороннее p-значение точного критерия Фишера: Точный критерий Фишера не может быть рассчитан для этих данных (должно быть по 2 уникальных значения)

Переменная: CO.1
Хи-квадрат статистика: 46.19345238095237
p-значение хи-квадрат: 0.7005595146800165
Одностороннее p-значение точного критерия Фишера: Точный критерий Фишера не может быть рассчитан для этих данных (должно быть по 2 уникаль

In [9]:
# Коэффициенты неопределенности для пар независимых переменных
pair_uncertainty = uncertainty_coefficients_multiple(y, X, k=2)
pair_uncertainty

,Variable 1,Variable 2,L
0,SBP.1,DBP.1,1.0


In [4]:
# Коэффициенты неопределенности для всех двух независимых переменных одновременно
all_uncertainty = uncertainty_coefficients_multiple(y, X, k=1)
all_uncertainty

,Variable 1,L
1,DBP.1,0.956155
0,SBP.1,0.912310
